In [1]:
import tensorflow as tf
import pandas as pd 
import tensorflow_io as tfio 
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import os
from sklearn.cluster import KMeans

In [2]:
main_dir=("/home/tkrsh/osic-main/")

files=[]

for dirname, _, filenames in os.walk(main_dir):
    for filename in (filenames):
        files.append(os.path.join((dirname), filename))

files=[x for x in files if '.csv' not in x]

train_images= [str(x) for x in files if 'train'  in x]
test_images = [str(x) for x in files if 'test'   in x]

In [58]:
def score(y_true, y_pred):
    tf.dtypes.cast(y_true, tf.float32)
    tf.dtypes.cast(y_pred, tf.float32)
    sigma = y_pred[:, 2] - y_pred[:, 0]
    fvc_pred = y_pred[:, 1]
    
    #sigma_clip = sigma + C1
    sigma_clip = tf.maximum(sigma, C1)
    delta = tf.abs(y_true[:, 0] - fvc_pred)
    delta = tf.minimum(delta, C2)
    sq2 = tf.sqrt( tf.dtypes.cast(2, dtype=tf.float32) )
    metric = (delta / sigma_clip)*sq2 + tf.math.log(sigma_clip* sq2)
    return K.mean(metric)

In [3]:
def decode_image(image_path):
    image_bytes = tf.io.read_file(image_path)
    image = tfio.image.decode_dicom_image(image_bytes, dtype=tf.uint16)
    image=np.squeeze(image.numpy())
    return image

def show_scan(image):
    img = decode_image(image)
    patient_name=str(image).split('/')[1]
    fig, ax = plt.subplots()
    im=ax.imshow(img,cmap='Greys')
    plt.axis('off')
    plt.title("Baseline CT Scan of Patient {}".format(patient_name))
    fig.set_size_inches(9,9)
    plt.show()

In [4]:
train=pd.read_csv(main_dir+"train.csv")
df=train.copy()

In [5]:
df.head()

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker


In [6]:
df=train.copy()

In [7]:
df=pd.concat([df,pd.get_dummies(df['Sex'])],axis=1).drop(['Sex'],axis=1)


In [8]:
df=pd.concat([df,pd.get_dummies(df['SmokingStatus'])],axis=1).drop(['SmokingStatus'],axis=1)


In [9]:
df.head()

,Patient,Weeks,FVC,Percent,Age,Female,Male,Currently smokes,Ex-smoker,Never smoked
0,ID00007637202177411956430,-4,2315,58.253649,79,0,1,0,1,0
1,ID00007637202177411956430,5,2214,55.712129,79,0,1,0,1,0
2,ID00007637202177411956430,7,2061,51.862104,79,0,1,0,1,0
3,ID00007637202177411956430,9,2144,53.950679,79,0,1,0,1,0
4,ID00007637202177411956430,11,2069,52.063412,79,0,1,0,1,0


In [10]:
df['dFVC'] =df["FVC"]-df["FVC"].shift(1)

In [11]:
df['d%'] = df["Percent"]-df["Percent"].shift(1)

In [12]:
df['Gap'] = df["Weeks"]-df["Weeks"].shift(1)

In [13]:
df.fillna(0,inplace=True)

In [14]:
df.head()

,Patient,Weeks,FVC,Percent,Age,Female,Male,Currently smokes,Ex-smoker,Never smoked,dFVC,d%,Gap
0,ID00007637202177411956430,-4,2315,58.253649,79,0,1,0,1,0,0.0,0.000000,0.0
1,ID00007637202177411956430,5,2214,55.712129,79,0,1,0,1,0,-101.0,-2.541520,9.0
2,ID00007637202177411956430,7,2061,51.862104,79,0,1,0,1,0,-153.0,-3.850025,2.0
3,ID00007637202177411956430,9,2144,53.950679,79,0,1,0,1,0,83.0,2.088576,2.0
4,ID00007637202177411956430,11,2069,52.063412,79,0,1,0,1,0,-75.0,-1.887267,2.0


In [15]:
df_1=df[df['Patient']=="ID00007637202177411956430"]

In [16]:
Means=KMeans(n_clusters=3).fit((df["Age"].values).reshape(-1,1))

In [17]:
df["Age_Cat"]=Means.labels_

In [18]:
df=pd.concat([df,pd.get_dummies(df['Age_Cat'],prefix="Age_Cat")],axis=1).drop(['Age_Cat'],axis=1)


In [19]:
df.head()

,Patient,Weeks,FVC,Percent,Age,Female,Male,Currently smokes,Ex-smoker,Never smoked,dFVC,d%,Gap,Age_Cat_0,Age_Cat_1,Age_Cat_2
0,ID00007637202177411956430,-4,2315,58.253649,79,0,1,0,1,0,0.0,0.000000,0.0,0,1,0
1,ID00007637202177411956430,5,2214,55.712129,79,0,1,0,1,0,-101.0,-2.541520,9.0,0,1,0
2,ID00007637202177411956430,7,2061,51.862104,79,0,1,0,1,0,-153.0,-3.850025,2.0,0,1,0
3,ID00007637202177411956430,9,2144,53.950679,79,0,1,0,1,0,83.0,2.088576,2.0,0,1,0
4,ID00007637202177411956430,11,2069,52.063412,79,0,1,0,1,0,-75.0,-1.887267,2.0,0,1,0


In [20]:
train_df=df.drop("Patient",axis=1)

In [21]:
df["Gap"]= [int(x) for x in df["Gap"]]

In [22]:
df.head()

,Patient,Weeks,FVC,Percent,Age,Female,Male,Currently smokes,Ex-smoker,Never smoked,dFVC,d%,Gap,Age_Cat_0,Age_Cat_1,Age_Cat_2
0,ID00007637202177411956430,-4,2315,58.253649,79,0,1,0,1,0,0.0,0.000000,0,0,1,0
1,ID00007637202177411956430,5,2214,55.712129,79,0,1,0,1,0,-101.0,-2.541520,9,0,1,0
2,ID00007637202177411956430,7,2061,51.862104,79,0,1,0,1,0,-153.0,-3.850025,2,0,1,0
3,ID00007637202177411956430,9,2144,53.950679,79,0,1,0,1,0,83.0,2.088576,2,0,1,0
4,ID00007637202177411956430,11,2069,52.063412,79,0,1,0,1,0,-75.0,-1.887267,2,0,1,0


In [23]:
sub=pd.read_csv(main_dir+"sample_submission.csv")

In [24]:
sub.head()

,Patient_Week,FVC,Confidence
0,ID00419637202311204720264_-12,2000,100
1,ID00421637202311550012437_-12,2000,100
2,ID00422637202311677017371_-12,2000,100
3,ID00423637202312137826377_-12,2000,100
4,ID00426637202313170790466_-12,2000,100


In [29]:
df.head()

,Patient,Weeks,FVC,Percent,Age,Female,Male,Currently smokes,Ex-smoker,Never smoked,dFVC,d%,Gap,Age_Cat_0,Age_Cat_1,Age_Cat_2
0,ID00007637202177411956430,-4,2315,58.253649,79,0,1,0,1,0,0.0,0.000000,0,0,1,0
1,ID00007637202177411956430,5,2214,55.712129,79,0,1,0,1,0,-101.0,-2.541520,9,0,1,0
2,ID00007637202177411956430,7,2061,51.862104,79,0,1,0,1,0,-153.0,-3.850025,2,0,1,0
3,ID00007637202177411956430,9,2144,53.950679,79,0,1,0,1,0,83.0,2.088576,2,0,1,0
4,ID00007637202177411956430,11,2069,52.063412,79,0,1,0,1,0,-75.0,-1.887267,2,0,1,0


In [30]:
from sklearn.model_selection import KFold

In [31]:
n_folds=5

In [35]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
from   tensorflow.keras import Sequential

In [116]:
z=x_train.shape

In [117]:
z

(1549, 13)

In [118]:
def regular_model(z):
    model= Sequential()
    model.add(L.Dense(400,input_shape=z))
    model.add(L.Dense(200,activation='relu'))
    model.add(L.Dense(64,activation='relu'))
    model.add(L.Dense(32,activation='relu'))
    model.add(L.Dense(1))
    model.compile(loss="mse",optimizer=tf.keras.optimizers.Adam(),metrics=[score])
    model.summary()
    return model

In [121]:
net=regular_model(z)

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_91 (Dense)             (None, 1549, 400)         5600      
_________________________________________________________________
dense_92 (Dense)             (None, 1549, 200)         80200     
_________________________________________________________________
dense_93 (Dense)             (None, 1549, 64)          12864     
_________________________________________________________________
dense_94 (Dense)             (None, 1549, 32)          2080      
_________________________________________________________________
dense_95 (Dense)             (None, 1549, 1)           33        
Total params: 100,777
Trainable params: 100,777
Non-trainable params: 0
_________________________________________________________________


In [127]:
df.head()

,Patient,Weeks,FVC,Percent,Age,Female,Male,Currently smokes,Ex-smoker,Never smoked,dFVC,d%,Gap,Age_Cat_0,Age_Cat_1,Age_Cat_2
0,ID00007637202177411956430,-4,2315,58.253649,79,0,1,0,1,0,0.0,0.000000,0,0,1,0
1,ID00007637202177411956430,5,2214,55.712129,79,0,1,0,1,0,-101.0,-2.541520,9,0,1,0
2,ID00007637202177411956430,7,2061,51.862104,79,0,1,0,1,0,-153.0,-3.850025,2,0,1,0
3,ID00007637202177411956430,9,2144,53.950679,79,0,1,0,1,0,83.0,2.088576,2,0,1,0
4,ID00007637202177411956430,11,2069,52.063412,79,0,1,0,1,0,-75.0,-1.887267,2,0,1,0


In [128]:
x_train.head()

,Weeks,FVC,Percent,Age,Female,Male,Currently smokes,Ex-smoker,Never smoked,dFVC,d%,Gap,Age_Cat_0,Age_Cat_1,Age_Cat_2
0,-4,2315,58.253649,79,0,1,0,1,0,0.0,0.000000,0,0,1,0
1,5,2214,55.712129,79,0,1,0,1,0,-101.0,-2.541520,9,0,1,0
2,7,2061,51.862104,79,0,1,0,1,0,-153.0,-3.850025,2,0,1,0
3,9,2144,53.950679,79,0,1,0,1,0,83.0,2.088576,2,0,1,0
4,11,2069,52.063412,79,0,1,0,1,0,-75.0,-1.887267,2,0,1,0


In [130]:
x_train=x_train.drop("FVC",axis=1)

In [131]:
y_train=df.pop("FVC")

In [132]:
y_train

0       2315
1       2214
2       2061
3       2144
4       2069
        ... 
1544    2712
1545    2978
1546    2908
1547    2975
1548    2774
Name: FVC, Length: 1549, dtype: int64

In [153]:
dataset = tf.data.Dataset.from_tensor_slices((x_train.values, y_train.values))


In [173]:
train_dataset=dataset.batch(9)

In [155]:
for feat, targ in dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))


Features: [-4.         58.25364872 79.          0.          1.          0.
  1.          0.          0.          0.          0.          0.
  1.          0.        ], Target: 2315
Features: [   5.           55.71212884   79.            0.            1.
    0.            1.            0.         -101.           -2.54151988
    9.            0.            1.            0.        ], Target: 2214
Features: [   7.           51.86210367   79.            0.            1.
    0.            1.            0.         -153.           -3.85002516
    2.            0.            1.            0.        ], Target: 2061
Features: [ 9.         53.95067942 79.          0.          1.          0.
  1.          0.         83.          2.08857574  2.          0.
  1.          0.        ], Target: 2144
Features: [ 11.          52.06341218  79.           0.           1.
   0.           1.           0.         -75.          -1.88726724
   2.           0.           1.           0.        ], Target: 2069


In [87]:
history= model.fit(train_x)

NameError: name 'train_x' is not defined

In [141]:
net.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_91 (Dense)             (None, 1549, 400)         5600      
_________________________________________________________________
dense_92 (Dense)             (None, 1549, 200)         80200     
_________________________________________________________________
dense_93 (Dense)             (None, 1549, 64)          12864     
_________________________________________________________________
dense_94 (Dense)             (None, 1549, 32)          2080      
_________________________________________________________________
dense_95 (Dense)             (None, 1549, 1)           33        
Total params: 100,777
Trainable params: 100,777
Non-trainable params: 0
_________________________________________________________________


In [62]:
df.head()

,Patient,Weeks,FVC,Percent,Age,Female,Male,Currently smokes,Ex-smoker,Never smoked,dFVC,d%,Gap,Age_Cat_0,Age_Cat_1,Age_Cat_2
0,ID00007637202177411956430,-4,2315,58.253649,79,0,1,0,1,0,0.0,0.000000,0,0,1,0
1,ID00007637202177411956430,5,2214,55.712129,79,0,1,0,1,0,-101.0,-2.541520,9,0,1,0
2,ID00007637202177411956430,7,2061,51.862104,79,0,1,0,1,0,-153.0,-3.850025,2,0,1,0
3,ID00007637202177411956430,9,2144,53.950679,79,0,1,0,1,0,83.0,2.088576,2,0,1,0
4,ID00007637202177411956430,11,2069,52.063412,79,0,1,0,1,0,-75.0,-1.887267,2,0,1,0


In [125]:
x_train=df.drop("Patient",axis=1)

In [ ]:
tf.kera

In [73]:
x_train=x_train.drop("Age",axis=1)

In [79]:
x_train=x_train.drop("FVC",axis=1)

In [80]:
x_train.head()

,Weeks,Percent,Female,Male,Currently smokes,Ex-smoker,Never smoked,dFVC,d%,Gap,Age_Cat_0,Age_Cat_1,Age_Cat_2
0,-4,58.253649,0,1,0,1,0,0.0,0.000000,0,0,1,0
1,5,55.712129,0,1,0,1,0,-101.0,-2.541520,9,0,1,0
2,7,51.862104,0,1,0,1,0,-153.0,-3.850025,2,0,1,0
3,9,53.950679,0,1,0,1,0,83.0,2.088576,2,0,1,0
4,11,52.063412,0,1,0,1,0,-75.0,-1.887267,2,0,1,0


In [190]:
def get_compiled_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
        
    tf.keras.layers.Dense(1)
    ])

    model.compile(optimizer='adam',
            loss="mse",
            metrics=["crossentropy"])
    return model

In [191]:
model = get_compiled_model()

In [192]:
model.fit(train_dataset,epochs=2000)

Epoch 1/2000

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

173/173 [==============================] - 0s 1ms/step - loss: 2218876.2500 - crossentropy: -40496.2695
Epoch 2/2000
173/173 [==============================] - 0s 1ms/step - loss: 595344.5000 - crossentropy: -41012.5156
Epoch 3/2000
173/173 [==============================] - 0s 1ms/step - loss: 530320.0625 - crossentropy: -41012.5156
Epoch 4/2000
173/173 [==============================] - 0s 1ms/step - loss: 484766.1250 - crossentropy: -41012.5156
Epoch 5/2000
173/173 [=====================

Epoch 65/2000
173/173 [==============================] - 0s 1ms/step - loss: 124511.2969 - crossentropy: -41012.5156
Epoch 66/2000
173/173 [==============================] - 0s 1ms/step - loss: 124774.1328 - crossentropy: -41012.5156
Epoch 67/2000
173/173 [==============================] - 0s 1ms/step - loss: 123159.4297 - crossentropy: -41012.5156
Epoch 68/2000
173/173 [==============================] - 0s 1ms/step - loss: 122957.6250 - crossentropy: -41012.5156
Epoch 69/2000
173/173 [==============================] - 0s 1ms/step - loss: 121904.3594 - crossentropy: -41012.5156
Epoch 70/2000
173/173 [==============================] - 0s 2ms/step - loss: 121604.3984 - crossentropy: -41012.5156
Epoch 71/2000
173/173 [==============================] - 0s 1ms/step - loss: 121219.0703 - crossentropy: -41012.5156
Epoch 72/2000
173/173 [==============================] - 0s 2ms/step - loss: 120116.7500 - crossentropy: -41012.5156
Epoch 73/2000
173/173 [==============================] - 0s 2ms/

Epoch 135/2000
173/173 [==============================] - 0s 2ms/step - loss: 84538.8828 - crossentropy: -41012.5156
Epoch 136/2000
173/173 [==============================] - 0s 2ms/step - loss: 84525.6562 - crossentropy: -41012.5156
Epoch 137/2000
173/173 [==============================] - 0s 1ms/step - loss: 83945.0312 - crossentropy: -41012.5156
Epoch 138/2000
173/173 [==============================] - 0s 2ms/step - loss: 83407.7500 - crossentropy: -41012.5156
Epoch 139/2000
173/173 [==============================] - 0s 2ms/step - loss: 83332.2422 - crossentropy: -41012.5156
Epoch 140/2000
173/173 [==============================] - 0s 2ms/step - loss: 81766.1797 - crossentropy: -41012.5156
Epoch 141/2000
173/173 [==============================] - 0s 2ms/step - loss: 82488.8984 - crossentropy: -41012.5156
Epoch 142/2000
173/173 [==============================] - 0s 1ms/step - loss: 81040.4531 - crossentropy: -41012.5156
Epoch 143/2000
173/173 [==============================] - 0s 2ms

173/173 [==============================] - 0s 1ms/step - loss: 56390.8086 - crossentropy: -41012.5156
Epoch 206/2000
173/173 [==============================] - 0s 1ms/step - loss: 55751.9727 - crossentropy: -41012.5156
Epoch 207/2000
173/173 [==============================] - 0s 1ms/step - loss: 54482.8984 - crossentropy: -41012.5156
Epoch 208/2000
173/173 [==============================] - 0s 1ms/step - loss: 56295.1484 - crossentropy: -41012.5156
Epoch 209/2000
173/173 [==============================] - 0s 2ms/step - loss: 55317.0469 - crossentropy: -41012.5156
Epoch 210/2000
173/173 [==============================] - 0s 2ms/step - loss: 55909.9648 - crossentropy: -41012.5156
Epoch 211/2000
173/173 [==============================] - 0s 2ms/step - loss: 54659.0508 - crossentropy: -41012.5156
Epoch 212/2000
173/173 [==============================] - 0s 1ms/step - loss: 53373.1016 - crossentropy: -41012.5156
Epoch 213/2000
173/173 [==============================] - 0s 1ms/step - loss: 5

173/173 [==============================] - 0s 2ms/step - loss: 37305.5977 - crossentropy: -41012.5156
Epoch 276/2000
173/173 [==============================] - 0s 2ms/step - loss: 35331.3633 - crossentropy: -41012.5156
Epoch 277/2000
173/173 [==============================] - 0s 1ms/step - loss: 35434.7734 - crossentropy: -41012.5156
Epoch 278/2000
173/173 [==============================] - 0s 2ms/step - loss: 36460.0586 - crossentropy: -41012.5156
Epoch 279/2000
173/173 [==============================] - 0s 2ms/step - loss: 35299.4922 - crossentropy: -41012.5156
Epoch 280/2000
173/173 [==============================] - 0s 2ms/step - loss: 37180.0625 - crossentropy: -41012.5156
Epoch 281/2000
173/173 [==============================] - 0s 1ms/step - loss: 34924.3516 - crossentropy: -41012.5156
Epoch 282/2000
173/173 [==============================] - 0s 2ms/step - loss: 36354.4961 - crossentropy: -41012.5156
Epoch 283/2000
173/173 [==============================] - 0s 2ms/step - loss: 3

...

173/173 [==============================] - 0s 2ms/step - loss: 11716.8027 - crossentropy: -41012.5156
Epoch 696/2000
173/173 [==============================] - 0s 1ms/step - loss: 9641.5244 - crossentropy: -41012.5156
Epoch 697/2000
173/173 [==============================] - 0s 2ms/step - loss: 7702.4287 - crossentropy: -41012.5156
Epoch 698/2000
173/173 [==============================] - 0s 1ms/step - loss: 15389.1455 - crossentropy: -41012.5156
Epoch 699/2000
173/173 [==============================] - 0s 1ms/step - loss: 16740.0742 - crossentropy: -41012.5156
Epoch 700/2000
173/173 [==============================] - 0s 1ms/step - loss: 14587.3936 - crossentropy: -41012.5156
Epoch 701/2000
173/173 [==============================] - 0s 1ms/step - loss: 10540.5957 - crossentropy: -41012.5156
Epoch 702/2000
173/173 [==============================] - 0s 2ms/step - loss: 11310.7617 - crossentropy: -41012.5156
Epoch 703/2000
173/173 [==============================] - 0s 2ms/step - loss: 888

173/173 [==============================] - 0s 2ms/step - loss: 10253.2236 - crossentropy: -41012.5156
Epoch 766/2000
173/173 [==============================] - 0s 1ms/step - loss: 6843.9717 - crossentropy: -41012.5156
Epoch 767/2000
173/173 [==============================] - 0s 1ms/step - loss: 5850.9526 - crossentropy: -41012.5156
Epoch 768/2000
173/173 [==============================] - 0s 1ms/step - loss: 6729.8257 - crossentropy: -41012.5156
Epoch 769/2000
173/173 [==============================] - 0s 2ms/step - loss: 6421.6094 - crossentropy: -41012.5156
Epoch 770/2000
173/173 [==============================] - 0s 1ms/step - loss: 10610.7520 - crossentropy: -41012.5156
Epoch 771/2000
173/173 [==============================] - 0s 1ms/step - loss: 10401.9453 - crossentropy: -41012.5156
Epoch 772/2000
173/173 [==============================] - 0s 2ms/step - loss: 10113.3525 - crossentropy: -41012.5156
Epoch 773/2000
173/173 [==============================] - 0s 2ms/step - loss: 7114.

173/173 [==============================] - 0s 2ms/step - loss: 8193.9043 - crossentropy: -41012.5156
Epoch 836/2000
173/173 [==============================] - 0s 1ms/step - loss: 19379.1934 - crossentropy: -41012.5156
Epoch 837/2000
173/173 [==============================] - 0s 1ms/step - loss: 18557.2520 - crossentropy: -41012.5156
Epoch 838/2000
173/173 [==============================] - 0s 1ms/step - loss: 11580.2275 - crossentropy: -41012.5156
Epoch 839/2000
173/173 [==============================] - 0s 1ms/step - loss: 7433.0596 - crossentropy: -41012.5156
Epoch 840/2000
173/173 [==============================] - 0s 1ms/step - loss: 6850.0034 - crossentropy: -41012.5156
Epoch 841/2000
173/173 [==============================] - 0s 1ms/step - loss: 7137.7251 - crossentropy: -41012.5156
Epoch 842/2000
173/173 [==============================] - 0s 1ms/step - loss: 5928.9380 - crossentropy: -41012.5156
Epoch 843/2000
173/173 [==============================] - 0s 1ms/step - loss: 6369.4

173/173 [==============================] - 0s 2ms/step - loss: 10797.3076 - crossentropy: -41012.5156
Epoch 906/2000
173/173 [==============================] - 0s 2ms/step - loss: 9102.7383 - crossentropy: -41012.5156
Epoch 907/2000
173/173 [==============================] - 0s 2ms/step - loss: 9182.4219 - crossentropy: -41012.5156
Epoch 908/2000
173/173 [==============================] - 0s 2ms/step - loss: 6426.3623 - crossentropy: -41012.5156
Epoch 909/2000
173/173 [==============================] - 0s 2ms/step - loss: 5461.7280 - crossentropy: -41012.5156
Epoch 910/2000
173/173 [==============================] - 0s 2ms/step - loss: 5596.8794 - crossentropy: -41012.5156
Epoch 911/2000
173/173 [==============================] - 0s 2ms/step - loss: 5391.2329 - crossentropy: -41012.5156
Epoch 912/2000
173/173 [==============================] - 0s 2ms/step - loss: 15631.9111 - crossentropy: -41012.5156
Epoch 913/2000
173/173 [==============================] - 0s 2ms/step - loss: 16811.6

KeyboardInterrupt: 

In [193]:
from knockknock import discord_sender


In [194]:
webhook_url = "https://discordapp.com/api/webhooks/737146056017575968/UzcX3__SbysSIjQ_O2BGK7ti1h-45yLIV893G6ha23cWntoV9bt3Tv7E1lMcnEu2GZ0m"


SyntaxError: unexpected EOF while parsing (<ipython-input-196-262f15a70b4e>, line 1)

In [164]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [160]:
len(train_dataset)

TypeError: object of type 'BatchDataset' has no len()

In [158]:
model.fit(train_dataset, epochs=15)


Epoch 1/15

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



ValueError: in user code:

    /home/tkrsh/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /home/tkrsh/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/tkrsh/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/tkrsh/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/tkrsh/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:543 train_step  **
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    /home/tkrsh/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/compil

In [123]:
y_train=df["FVC"].values

In [78]:
y_train

array([2315, 2214, 2061, ..., 2908, 2975, 2774])

In [66]:
from sklearn.preprocessing import normalize

In [67]:
x_train=normalize(x_train)

In [124]:
y_train

array([2315, 2214, 2061, ..., 2908, 2975, 2774])

In [135]:
history=net.fit(train_dataset)

TypeError: 'NoneType' object is not callable

In [25]:
from sklearn.model_selection import KFold
import tensorflow as tf

In [482]:
# PARAMETERS
n_folds=5

In [473]:
def score(y_true, y_pred):
    tf.dtypes.cast(y_true, tf.float32)
    tf.dtypes.cast(y_pred, tf.float32)
    sigma = y_pred[:, 2] - y_pred[:, 0]
    fvc_pred = y_pred[:, 1]
    sigma_clip = tf.maximum(sigma, C1)
    delta = tf.abs(y_true[:, 0] - fvc_pred)
    delta = tf.minimum(delta, C2)
    sq2 = tf.sqrt( tf.dtypes.cast(2, dtype=tf.float32) )
    metric = (delta / sigma_clip)*sq2 + tf.math.log(sigma_clip* sq2)
    return K.mean(metric)

In [483]:
fold=KFold(n_splits=n_folds)

In [488]:
for train , test in fold.split(df):
  

SyntaxError: unexpected EOF while parsing (<ipython-input-488-2996885ce8b2>, line 2)